# AI Labor HSKA: Reinforcement Learning

## Wiederholung: Q-Learning

Q-Learning basiert auf Temporalen Differenz (TD)-Lernen (engl. Temporal Difference Learning). Die Idee von TD Methoden ist es, die bisherige Schätzung der Value-Function zusammen mit dem unmittelbar erhaltenen Reward nach Ausführung einer Aktion zu nutzen, um die bisherige Schätzung der Value-Function zu aktualisieren. Diese Aktualisierung findet nach jeder Aktionswahl statt. Im einfachsten Fall wählt der Agent eine Aktion anhand seiner Policy, erhält eine Belohnung und passt die Value-Function wie folgt an:

$$V(s_t) \leftarrow V(s_t) + \alpha [\overbrace{\underbrace{R_t + \gamma V(s_{t+1})}_{target} - V(s_t)}^{TD\ error}]$$

Daraus ergibt sich die Lernregel für das Q-Learning:

$$Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha\left[R(s_t,a_t)+\gamma \max_{a\in A(s_{t+1})} Q(s_{t+1},a) - 
Q(s_t,a_t)\right]$$

$Q$ approximiert dabei $q^*$ und damit die optimale Action-Value Function. Ist $q^*$ bekannt, können optimale Aktionen ohne jegliches Wissen über Nachfolgezustände (Successor States) oder deren Werte gewählt werden, da bereits die zu wählende Aktion für einen Zustand gegeben ist. Das heißt, der agent muss nichts über die Dynamik (Transition Model, Reward Function) der Umgebung wissen.

## Aufgabe 1

Vervollständige die in **agent.py** vorgebene Implementierung eines Agent, der mit Hilfe von Q-Learning eine optimale Policy $\pi^*$ findet. Pro Episode sind maximal 200 Steps in den beiden Ausprägungen der GridWorld erlaubt, danach wird das Environment zurückgesetzt.

Der Agent soll über 50 Episoden hinweg versuchen eine optimale Policy zu finden.

In [ ]:
%run ../setup.ipynb

In [ ]:
from lib import GridWorld
from lib.statistics import plot

n_episodes = 500
max_steps = 200

def interact_with_environment(env, agent, verbose=False):
    statistics = []
    if verbose:
        print('Startposition:')
        env.render()
    
    for episode in range(n_episodes):
        done = False
        total_reward = 0
        state = env.reset()
        #print("State 0 in Episode:",episode, state)
        
        for t in range(max_steps):
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            #print("State",t,"in Episode",episode,":", next_state)
            agent.train((state, action, next_state, reward, done))
            state = next_state
            total_reward += reward
            
            if done:
                break
        
        if verbose:
            print(f'episode: {episode}/{n_episodes}, score: {total_reward}, steps:{t}, e:{agent.epsilon:.2f}')
            env.render()
            
        statistics.append({
            'episode': episode,
            'score': total_reward,
            'steps': t
        })
        
    return statistics

### Aufgabe 1.1

Der Agent soll folgende Ausprägung der GridWorld lösen: Die Start- und Zielposition des Agenten ist fest vorgegeben und ändert sich nicht. Das Transition Model dieser GridWorld ist deterministisch (`transition_probability=1.0`). D.h. die vom Agenten gewählte Aktion wird in jedem Fall ausgeführt.

In [ ]:
env = GridWorld(env_x=8, env_y=8, init_agent_pos=(0,0), goal_pos=(7,7), max_steps=max_steps, transition_probability=1.0)

In [ ]:
from agent import QLearning

# Hyperparams
gamma = 0.75
epsilon = 0.001
epsilon_decay = 0.9
epsilon_min = 0.0001
alpha = 1.0
alpha_decay = 0.999
alpha_min = 0.001

agent = QLearning(action_dim=env.action_dim, state_dim=env.state_dim,
                  gamma=gamma, epsilon=epsilon, alpha=alpha)

statistics = interact_with_environment(env, agent, verbose=False)
plot(statistics)

### Aufgabe 1.2

Erweitere (falls nötig) die Implementierung deines Agenten, sodass er die RandomGridWorld (`transition_probability=0.8`) löst.

In [ ]:
env = GridWorld(env_x=8, env_y=8, init_agent_pos=None, goal_pos=None, max_steps=max_steps, transition_probability=0.99)
#transition_probability high = less random

In [ ]:
from agent import QLearning

# Hyperparams
gamma = 0.75
epsilon = 0.001
#epsilon_decay = 0.9
#epsilon_min = 0.0001
alpha = 0.9
#alpha_decay = 0.999
#alpha_min = 0.001

agent = QLearning(action_dim=env.action_dim, state_dim=env.state_dim,
                  gamma=gamma, epsilon=epsilon, alpha=alpha)

statistics = interact_with_environment(env, agent, verbose=False)
plot(statistics)

## Aufgabe 2
Beschreibe was eine optimale Policy ausmacht und begründe warum es mehrere optimale Policies gibt.

_Jede Policy, die das GridWorld mit der minimalen Anzahl Schritte durchläuft ist eine optimale Policy. Die minimale Anzahl an Schritten ist (für diese Art von Umgebungen) $N_y + N_x - 2$ (Breite + Höhe - Startzustand - Endzustand)._